In [1]:
import warnings

warnings.filterwarnings(action='ignore')

# Redactor Tutorial Code

## Epileptic Seizure Recognition (ESR) Dataset

In [2]:
from preprocessing import Dataset
import numpy as np

np.random.seed(227)

test_size = 0.25
n_target = 10

ESR = Dataset('ESR')
target_indices = np.random.choice(ESR.data.index, n_target)
#target_indices = np.array(['i10751','i1076','i11425','i2452','i2821','i3984','i6338','i7893','i7917','i8775'])
(x_tr,y_tr), (x_te,y_te), (x_ta,y_ta), tr_scaler = ESR.split_dataset(test_size, target_indices)

In [3]:
target_indices

array(['i2016', 'i7016', 'i6123', 'i878', 'i10148', 'i9144', 'i11401',
       'i2976', 'i5932', 'i159'], dtype=object)

In [4]:
from model import SurrogateModels
    
model_names = ['nn_relu_50_25','nn_relu_200_100', 'nn_relu_25_10', 'svm_rbf', 'rf_gini']

s_models = SurrogateModels(model_names)
s_models.train_all(x_tr, y_tr)
s_models.show_performance([(x_tr,y_tr), (x_te,y_te), (x_ta,y_ta)],
                         cnames=['train', 'test','target'],
                         ptype='acc')
#s_models.cross_validation(x_tr, y_tr, k=3)

train models..


100%|██████████| 5/5 [03:48<00:00, 45.76s/it]
5it [00:15,  3.08s/it]


,train acc,test acc,target acc
s-nn_relu_50_25,0.9300,0.6706,0.9
s-nn_relu_200_100,1.0000,0.7221,0.9
s-nn_relu_25_10,0.7801,0.6379,0.7
s-svm_rbf,0.5885,0.5576,0.3
s-rf_gini,0.6262,0.5565,0.7


In [5]:
from prob_decision_boundary import PDB

prob_dec = PDB(s_models.models, y_dim = len(np.unique(y_te)))
x_all = np.concatenate([x_tr, x_te], axis=0)
prob_dec.fit_all(x_all)
sn_te_labels = prob_dec.predict(x_te)

100%|██████████| 5/5 [00:08<00:00,  1.75s/it]


In [6]:
sn_te_labels[sn_te_labels == -1] = 0
te_acc = sum(sn_te_labels==y_te)/len(y_te)

# Candidate Generation
## GAN-based

In [7]:
from gen_disinfos import GANcandidates

esr = ESR.data
column_cat = ESR.column_cat
column_int = ESR.column_int
columns_1hot = ESR.data_1hot.columns

gan_gen = GANcandidates()
gan_gen.fit(esr, column_cat, column_int)

In [8]:
_ = gan_gen.generate()
gan_cand_list = gan_gen.nearest_points(tr_scaler, target_indices, columns_1hot)

In [9]:
from IPython.display import display
import pandas as pd

display(pd.concat([g.iloc[[1]]for g in gan_cand_list],ignore_index=True))

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X169,X170,X171,X172,X173,X174,X175,X176,X177,X178
0,-257,407,280,-158,-118,-331,-85,96,-207,-75,...,101,11,518,-87,-787,-151,114,190,22,1172
1,-56,42,-112,81,-555,123,-149,93,49,-63,...,-54,474,-149,140,302,574,-1199,271,-567,-241
2,-1200,-195,-77,-266,-33,520,342,-169,-172,-167,...,-29,29,-406,219,421,142,-1422,-129,-3,-500
3,232,-128,303,222,221,117,138,30,-942,116,...,187,197,-50,-256,-371,-61,63,19,-491,204
4,-29,-112,286,-38,52,-26,77,-83,768,59,...,40,7,-44,1,350,-52,-119,-10,-10,197
5,-257,407,280,-158,-118,-331,-85,96,-207,-75,...,101,11,518,-87,-787,-151,114,190,22,1172
6,-257,407,280,-158,-118,-331,-85,96,-207,-75,...,101,11,518,-87,-787,-151,114,190,22,1172
7,115,97,760,74,-6,-79,132,-95,-95,40,...,-69,-435,92,46,-23,-490,16,-114,-60,568
8,-257,407,280,-158,-118,-331,-85,96,-207,-75,...,101,11,518,-87,-787,-151,114,190,22,1172
9,115,97,760,74,-6,-79,132,-95,-95,40,...,-69,-435,92,46,-23,-490,16,-114,-60,568


## WM-based

In [10]:
from gen_disinfos import WMcandidates, agg_disinfo
from tqdm import tqdm
  
esr_1hot = ESR.data_1hot
esr_label = ESR.label

wm_gen = WMcandidates(esr_1hot, esr_label, target_indices)
wm_cand_list = wm_gen.watermarking(tr_scaler, esr.columns, column_cat, column_int)

100%|██████████| 10/10 [01:06<00:00,  6.63s/it]


In [11]:
display(pd.concat([w.iloc[[0]]for w in wm_cand_list],ignore_index=True))

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X169,X170,X171,X172,X173,X174,X175,X176,X177,X178
0,-16,17,49,88,117,129,118,71,24,-37,...,-2,-6,-4,0,-19,-28,-34,-49,-53,-57
1,-1,1,-1,5,-1,-3,-3,3,11,16,...,20,7,3,10,13,13,23,28,26,13
2,-96,-62,-53,-23,5,3,-8,-32,-42,-61,...,-57,-54,-56,-49,-20,-8,-9,-14,-31,-37
3,81,136,160,117,62,-9,-71,-109,-92,-34,...,862,528,232,48,-55,-100,-107,-88,-77,-86
4,29,45,48,38,14,-14,-51,-72,-70,-52,...,-27,-19,-17,-10,-10,-31,-58,-85,-97,-105
5,-13,-4,3,11,3,-8,-10,0,10,10,...,25,21,19,18,15,-1,-20,-26,-29,-29
6,19,45,67,80,84,84,81,89,106,128,...,-2,4,4,14,11,3,-8,-19,-29,-40
7,-20,-19,-18,-6,10,19,25,26,26,12,...,-102,-98,-94,-100,-101,-109,-103,-94,-80,-70
8,-27,-23,-16,-13,-15,-10,-8,-10,-11,-16,...,30,36,33,32,32,30,31,36,41,37
9,91,13,-88,-196,-267,-269,-232,-179,-132,-94,...,-137,-109,-85,-63,-42,-30,-9,13,35,45


In [12]:
x_dis, y_dis = [], []
for ti in range(n_target):
    xt, yt = x_ta[ti], y_ta[ti]
    wm_cand = wm_cand_list[ti]
    gan_cand = gan_cand_list[ti]
    candidates = pd.concat((wm_cand, gan_cand))
    
    x_tmp, y_tmp = agg_disinfo(prob_dec, candidates, tr_scaler, x_tr, y_tr, xt, yt, 
                               columns_1hot, n_disinfo=50, alpha=0.7)
    x_dis.extend(x_tmp)
    y_dis.extend(y_tmp)

100%|██████████| 5/5 [00:00<00:00, 161.57it/s]


# Insert Disinformation

In [13]:
from model import VictimModels

model_names = ['nn_tanh_5_2', 'nn_relu_50_25', 'nn_relu_200_100', 'nn_relu_25_10', 'tree_gini', 'rf_gini', 'rf_entropy']

v_models = VictimModels(model_names)
v_models.train_all(x_tr, y_tr)
result_clean = v_models.show_performance([(x_tr,y_tr), (x_te,y_te), (x_ta,y_ta)],
                                         cnames=['train', 'test','target'],
                                         ptype='acc')
result_clean

train models..


100%|██████████| 7/7 [05:09<00:00, 44.16s/it]
7it [00:00, 14.34it/s]


,train acc,test acc,target acc
v-nn_tanh_5_2,0.4611,0.4073,0.6
v-nn_relu_50_25,0.9300,0.6706,0.9
v-nn_relu_200_100,1.0000,0.7221,0.9
v-nn_relu_25_10,0.7801,0.6379,0.7
v-tree_gini,0.5091,0.4463,0.6
v-rf_gini,0.6262,0.5565,0.7
v-rf_entropy,0.5920,0.5252,0.7


In [14]:
x_tr_dis = np.concatenate((x_tr, x_dis), axis=0)
y_tr_dis = np.concatenate((y_tr, np.array(y_dis)[:,0]), axis=0).astype(int)

In [15]:
v_models_dis = VictimModels(model_names)
v_models_dis.train_all(x_tr_dis, y_tr_dis)
result_dis = v_models_dis.show_performance([(x_tr,y_tr), (x_te,y_te), (x_ta,y_ta)],
                                         cnames=['train', 'test','target'],
                                         ptype='acc')

train models..


100%|██████████| 7/7 [04:27<00:00, 38.23s/it]
7it [00:00, 14.21it/s]


In [16]:
from utils import compare_result

compare_result(result_clean, result_dis)

,mean,std
train acc,-0.037143,1.486323
test acc,0.294286,1.047391
target acc,-34.285714,9.759001


In [17]:
r1 = v_models.show_performance([(x_tr,y_tr), (x_te,y_te), (x_ta,y_ta)],
                                 cnames=['train', 'test','target'],
                                 ptype='y_conf')
r2 = v_models_dis.show_performance([(x_tr,y_tr), (x_te,y_te), (x_ta,y_ta)],
                                 cnames=['train', 'test','target'],
                                 ptype='y_conf')
compare_result(r1, r2)

7it [00:00, 16.21it/s]
7it [00:00, 16.50it/s]


,mean,std
train y_conf,-0.470000,0.406079
test y_conf,-0.190000,0.569883
target y_conf,-17.084286,14.327076


# Membership Inference Attack

In [19]:
from sklearn.model_selection import train_test_split
from model import AttackModels, attack_input

np.random.seed(726)

vi = 1
victim_clean = v_models.models[vi]
x_mia, y_mia, x_mia_ta, y_mia_ta = attack_input(victim_clean, x_tr, y_tr, x_te, y_te, x_ta, y_ta)
x_mia_tr, x_mia_te, y_mia_tr, y_mia_te = train_test_split(x_mia, y_mia, test_size  = 0.25)

In [20]:
model_names = ['nn_tanh_5_2','nn_relu_5_2', 'nn_identity', 'tree_gini', 'tree_entropy',
               'rf_gini', 'rf_entropy', 'ada', 'log_reg']

a_models = AttackModels()
a_models.train_all(x_mia_tr, y_mia_tr)
a_result_clean = a_models.show_performance([(x_mia_tr,y_mia_tr), (x_mia_te,y_mia_te), (x_mia_ta,y_mia_ta)],
                         cnames=['train attack', 'test attack','target attack'])
a_result_clean

train models..


100%|██████████| 19/19 [00:20<00:00,  1.10s/it]
19it [00:02,  7.14it/s]


,train attack acc,test attack acc,target attack acc
a-nn_tanh_5_2,0.6387,0.6127,0.5
a-nn_tanh_10_2,0.6396,0.6210,0.5
a-nn_relu_5_2,0.6375,0.6189,0.5
a-nn_relu_50_25,0.6389,0.6266,0.5
a-nn_relu_200_100,0.6398,0.6196,0.5
a-nn_relu_25_10,0.6371,0.6259,0.5
a-nn_log_5_2,0.6345,0.6245,0.5
a-nn_identity,0.6387,0.6147,0.5
a-tree_gini,0.6654,0.6008,0.7
a-tree_entropy,0.6665,0.6015,0.7


In [21]:
victim_dis = v_models_dis.models[vi]
x_mia, y_mia, x_mia_ta, y_mia_ta = attack_input(victim_dis, x_tr, y_tr, x_te, y_te, x_ta, y_ta)
x_mia_tr, x_mia_te, y_mia_tr, y_mia_te = train_test_split(x_mia, y_mia, test_size  = 0.25)

In [22]:
a_models_dis = AttackModels(model_names)
a_models_dis.train_all(x_mia_tr, y_mia_tr)
a_result_dis = a_models_dis.show_performance([(x_mia_tr,y_mia_tr), (x_mia_te,y_mia_te), (x_mia_ta,y_mia_ta)],
                         cnames=['train attack', 'test attack','target attack'])
a_result_dis

train models..


100%|██████████| 9/9 [00:05<00:00,  1.55it/s]
9it [00:00, 35.70it/s]


,train attack acc,test attack acc,target attack acc
a-nn_tanh_5_2,0.6324,0.6099,0.6
a-nn_relu_5_2,0.6259,0.6092,0.6
a-nn_identity,0.6255,0.6078,0.6
a-tree_gini,0.6575,0.5974,0.5
a-tree_entropy,0.6718,0.5737,0.6
a-rf_gini,0.6674,0.5974,0.5
a-rf_entropy,0.6716,0.5987,0.5
a-ada,0.6477,0.5994,0.6
a-log_reg,0.6252,0.6071,0.6


In [25]:
idxs = a_result_clean['target attack acc'] > 0.5
compare_result(a_result_clean.loc[idxs], a_result_dis.loc[idxs])

,mean,std
train attack acc,-0.385,0.614139
test attack acc,-1.770,1.034569
target attack acc,-17.500,5.000000
